## Importações e Configuração Inicial
 - AJUSTAR PARA BUSCAR INFORMAÇÕES DE PONTUAÇÕES MENSAIS

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
from pathlib import Path
from bs4 import BeautifulSoup
import json, re
import unicodedata
import time
import requests

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2026-01-26 08:32:54,188 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
from bs4 import BeautifulSoup
from pathlib import Path
import json, re
import unicodedata

# ajuste se precisar
HTML_PATH = Path("pagina_liga_classica.html")

def clean_name(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t\r\n\f\v]+", " ", s).strip()
    return s

# carrega html
html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# 1) principais seletores desta vers?o da p?gina
seletores = [
    "div.info-time h3",              # novo layout (print do DevTools)
    # "a.button-container-card h3",    # h3 dentro do link do time
    # "div.card-row-info h3",          # varia?es
    # "section.container-lista-times h3",
    # "ranking-de-liga h3",
]

nomes = []
for sel in seletores:
    for el in soup.select(sel):
        t = clean_name(el.get_text(" ", strip=True))
        if t:
            nomes.append(t)

# 2) fallback: pegar o texto do link que aponta para #!/time/<id>
if not nomes:
    for a in soup.select("a[href*='#!/time/']"):
        t = clean_name(a.get_text(" ", strip=True))
        if t:
            nomes.append(t)

# 3) fallback final: regex focada no bloco info-time -> <h3>...</h3>
if not nomes:
    for m in re.findall(r'class="info-time"[^>]*>.*?<h3[^>]*>(.*?)</h3>', html, flags=re.I|re.S):
        t = BeautifulSoup(m, "html.parser").get_text(" ", strip=True)
        t = clean_name(t)
        if t:
            nomes.append(t)

# dedup (ordem de apari?o) + ordena?o alfab?tica
visto, times = set(), []
for n in nomes:
    if n and n not in visto:
        visto.add(n)
        times.append(n)

times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times[:10])  # amostra

# salva snippet JS ao lado do HTML
OUT_JS = HTML_PATH.with_name("participantesLiga_Classica.js")
OUT_JS.write_text(
    "window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
    encoding="utf-8"
)
print(f"Snippet salvo em: {OUT_JS}")


52 times encontrados
['Acessar o perfil do time A Lenda Super Vasco F.c', 'Acessar o perfil do time A Lenda Super Vascão f.c', 'Acessar o perfil do time AZURRA82', 'Acessar o perfil do time Bandoleros FCS', 'Acessar o perfil do time BordonFC04', 'Acessar o perfil do time C.A. Charru@', 'Acessar o perfil do time cartola scheuer17', 'Acessar o perfil do time CARTOLEIRO DO VALLE PRO26.5', 'Acessar o perfil do time dasdoresfc', 'Acessar o perfil do time DM Studio']
Snippet salvo em: participantesLiga_Classica.js


In [3]:
# ? ajuste aqui se precisar
HTML_PATH = Path("pagina_liga_classica.html")  

def clean_name(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t\r\n\f\v]+", " ", s).strip()
    return s

# carrega html
soup = BeautifulSoup(HTML_PATH.read_text(encoding="utf-8", errors="ignore"), "html.parser")

# coleta nomes (principal + fallback)
nomes = [clean_name(el.get_text(" ", strip=True)) for el in soup.select("p.nome-time") if el]
if not nomes:
    nomes = [
        clean_name((a.get("title") or a.get_text(" ", strip=True)))
        for a in soup.select("a.link-perfil-time")
        if (a.get("title") or a.get_text(strip=True))
    ]

# dedup + ordena alfab?tico
visto, times = set(), []
for n in nomes:
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML
OUT_JS = HTML_PATH.with_name("participantesLiga_Classica.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n", encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


51 times encontrados
['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'AZURRA82', 'Bandoleros FCS', 'BordonFC04', 'C.A. Charru@', 'cartola scheuer17', 'CARTOLEIRO DO VALLE PRO26.5', 'dasdoresfc', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'FBC Colorado II', 'FC castelo Branco 2', 'FC Los Castilho', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'FÚRIA LEON', 'Gig@ntte', 'Gremiomaniasm', 'Grêmio imortal 37', 'JUV. KP', 'JV5 Tricolor Gaúcho', 'LISI GREMISTA', 'lsauer fc', 'Luis lemes inter', 'MAFRA MARTINS FC', 'Mau Humor F.C.', 'mercearia Estrela', 'Máquina Laranjja', 'NaoVaiDescer!', 'Paulo Virgili FC', 'Pity10', 'pra sempre imortal fc', 'Profit Soccer', 'PUXE FC', 'Rolo Compressor ZN', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua PB1', 'TATITTA FC', 'Tatols Beants F.C', 'teves_futsal20 f.c', 'Texas Club 2026', 'TIGRE LEON', 'Time do S.A.P.O', 'VASCO MARTINS FC']
Snippet salvo em: participantesLiga_Classi

In [4]:
JS_PATH = Path("participantesLiga_Classica.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times[:10]

51 times


['A Lenda Super Vasco F.c',
 'A Lenda Super Vascão f.c',
 'AZURRA82',
 'Bandoleros FCS',
 'BordonFC04',
 'C.A. Charru@',
 'cartola scheuer17',
 'CARTOLEIRO DO VALLE PRO26.5',
 'dasdoresfc',
 'DM Studio']

In [5]:

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

print(nomes_times)

['A Lenda Super Vasco F.c', 'A Lenda Super Vascão f.c', 'AZURRA82', 'Bandoleros FCS', 'BordonFC04', 'C.A. Charru@', 'cartola scheuer17', 'CARTOLEIRO DO VALLE PRO26.5', 'dasdoresfc', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'FBC Colorado II', 'FC castelo Branco 2', 'FC Los Castilho', 'Fedato Futebol Clube', 'FIGUEIRA DA ILHA', 'FÚRIA LEON', 'Gig@ntte', 'Gremiomaniasm', 'Grêmio imortal 37', 'JUV. KP', 'JV5 Tricolor Gaúcho', 'LISI GREMISTA', 'lsauer fc', 'Luis lemes inter', 'MAFRA MARTINS FC', 'Mau Humor F.C.', 'mercearia Estrela', 'Máquina Laranjja', 'NaoVaiDescer!', 'Paulo Virgili FC', 'Pity10', 'pra sempre imortal fc', 'Profit Soccer', 'PUXE FC', 'Rolo Compressor ZN', 'S.E.R. GRILLO', 'seralex', 'SERGRILLO', 'Sport Clube PAIM', 'Super Vasco f.c', 'SUPER VASCÃO F.C', 'Tabajara de Inhaua PB1', 'TATITTA FC', 'Tatols Beants F.C', 'teves_futsal20 f.c', 'Texas Club 2026', 'TIGRE LEON', 'Time do S.A.P.O', 'VASCO MARTINS FC']


In [6]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_classica.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)


✅ Arquivo salvo com sucesso: links_times_cartola_liga_classica.xlsx


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,A Lenda Super Vascão f.c,49355335,https://cartola.globo.com/#!/time/49355335
2,AZURRA82,18346776,https://cartola.globo.com/#!/time/18346776
3,Bandoleros FCS,13913874,https://cartola.globo.com/#!/time/13913874
4,BordonFC04,50988641,https://cartola.globo.com/#!/time/50988641
5,C.A. Charru@,25748736,https://cartola.globo.com/#!/time/25748736
6,cartola scheuer17,3851966,https://cartola.globo.com/#!/time/3851966
7,CARTOLEIRO DO VALLE PRO26.5,49243759,https://cartola.globo.com/#!/time/49243759
8,dasdoresfc,7017989,https://cartola.globo.com/#!/time/7017989
9,DM Studio,387186,https://cartola.globo.com/#!/time/387186


In [7]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual
# rodada_atual = 1

print("Rodada Atual:", rodada_atual)

Rodada Atual: 1


## Obter Pontuações da Rodada

In [8]:
# ===============================
#  CARTOLA FC - CLASSIFICAÇÃO COM PARCIAIS DA RODADA
# ===============================
import time
import json
import requests
import pandas as pd

# Se usar exceptions específicas da lib:
try:
    import cartolafc
    from cartolafc import CartolaFCError
except Exception:
    CartolaFCError = Exception  # fallback

# -------------------------------
# CONFIG
# -------------------------------
CAP_MULT = 1.5        # multiplicador do capitão (Cartola = dobra pontos)
PER_REQ_SLEEP = 0.9   # pausa entre requests (evitar 429/403)
MAX_RETRIES = 3       # tentativas por requisição

# Sessão HTTP
sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

# -------------------------------
# HELPERS HTTP / API
# -------------------------------
def http_status_e_rodada():
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    data = r.json()
    return int(data.get("status_mercado", 0)), int(data.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    data = r.json()
    return data.get("atletas", {})

def get_escalacao_por_rodada_http_dbg(time_id: int, rodada: int, max_retries=MAX_RETRIES):
    """
    Busca escalação travada na rodada; se não vier, cai para 'última escalação conhecida'.
    Retorna: (atletas:list, capitao_id:int|None, url_usada:str|None, http_status:int|None, motivo:str|None)
    """
    urls = [
        f"https://api.cartola.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartola.globo.com/time/id/{time_id}",
    ]
    last_err = None
    for url in urls:
        wait = 0.7
        for attempt in range(1, max_retries + 1):
            try:
                r = sess.get(url, timeout=30)
                status = r.status_code
                if status == 200:
                    d = r.json()
                    atletas = d.get("atletas") or []
                    capitao_id = d.get("capitao_id")
                    if atletas:
                        return atletas, capitao_id, url, status, None  # ok
                    else:
                        return [], d.get("capitao_id"), url, status, "sem_atletas"
                elif status in (403, 429, 503):
                    last_err = f"status={status}"
                    time.sleep(wait); wait *= 1.8
                    continue
                else:
                    last_err = f"status={status}"
                    break
            except Exception as e:
                last_err = f"exc:{type(e).__name__}"
                time.sleep(wait); wait *= 1.8
        # tenta próxima URL (fallback)
    return [], None, None, None, last_err  # falhou nas duas

# -------------------------------
# 1) RODADA ATUAL E DF BASE
# -------------------------------
try:
    rodada_atual = api.mercado().rodada_atual
except CartolaFCError:
    rodada_atual = 1

# rodada_atual = 38

def campeonato_comecou(ids_times: dict) -> bool:
    # considera "começou" se já existe pontuação na rodada 1 para algum time
    for time_id in ids_times.values():
        try:
            p = api.time(time_id=int(time_id), rodada=1).ultima_pontuacao
            if p is not None:
                return True
        except Exception:
            continue
    return False

if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
    total_rodadas = 38
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(),
                                 columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = 0.0
else:
    def obter_pontuacao_por_rodada(time_id: int, rodada_atual_api: int) -> dict:
        # Agora: 1 .. rodada_atual_api (INCLUSIVO)
        pontuacoes = {}
        fim_inclusivo = max(1, int(rodada_atual_api))
        for rodada in range(1, fim_inclusivo + 1):
            try:
                t = api.time(time_id=int(time_id), rodada=rodada)
                pontuacoes[rodada] = t.ultima_pontuacao
            except Exception:
                pontuacoes[rodada] = None
        return pontuacoes


    df_pontuacoes = pd.DataFrame()
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(int(time_id), rodada_atual)
        df_pontuacoes[nome] = pd.Series(pontuacoes)

    # df_pontuacoes = df_pontuacoes.transpose()
    # if rodada_atual > 1:
    #     df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]

    df_pontuacoes = df_pontuacoes.transpose()
    if rodada_atual > 1:
        df_pontuacoes.columns = [f'Rodada {i}' for i in range(1, rodada_atual + 1)]

    df_pontuacoes = df_pontuacoes.apply(pd.to_numeric, errors="coerce").fillna(0.0)

# -------------------------------
# 2) APLICA PARCIAIS (SOMENTE SE status==2)
# -------------------------------
status_http, rodada_http = http_status_e_rodada()
rodada_parcial = int(rodada_http) if status_http == 2 else None

print(
    f"Status HTTP={status_http} | rodada_http={rodada_http} | "
    f"rodada_api={rodada_atual} | rodada_parcial={rodada_parcial}"
)

if rodada_parcial is not None:
    col_parcial = f"Rodada {rodada_parcial}"
    if col_parcial not in df_pontuacoes.columns:
        df_pontuacoes[col_parcial] = 0.0  # cria coluna da rodada em andamento

    # remove marcador para evitar conflito
    if "Lider_Rodada" in df_pontuacoes.index:
        df_pontuacoes = df_pontuacoes.drop(index="Lider_Rodada", errors="ignore")

    print("🟡 Rodada em andamento — consultando parciais…")
    parciais_map = get_parciais()
    if not parciais_map:
        print("⚠️ Endpoint de parciais retornou vazio agora. Mantendo valores anteriores.")
    else:
        atualizados, relatorio = 0, []
        for nome_time, time_id in ids_times.items():
            try:
                time_id = int(time_id)
                atletas, capitao_id, url_usada, http_st, motivo = \
                    get_escalacao_por_rodada_http_dbg(time_id, rodada_parcial)

                # fallback via lib quando vier sem atletas
                if not atletas and (motivo in ("sem_atletas", "sem_escalacao", None)):
                    try:
                        t = api.time(time_id=time_id, rodada=rodada_parcial)
                        atletas = [{"atleta_id": a.atleta_id} for a in (t.atletas or [])]
                        if not capitao_id:
                            capitao_id = getattr(t, "capitao_id", None)
                        http_st = http_st or 200
                    except Exception:
                        pass

                total, matched = 0.0, 0
                for a in (atletas or []):
                    aid = str(a.get("atleta_id"))
                    if aid in parciais_map:
                        p = float(parciais_map[aid].get("pontuacao", 0) or 0)
                        if capitao_id and a.get("atleta_id") == capitao_id:
                            p *= CAP_MULT
                        total += p; matched += 1

                df_pontuacoes.loc[nome_time, col_parcial] = round(total, 2)
                atualizados += 1
                relatorio.append((nome_time, http_st or 200, len(atletas or []), matched, round(total, 2), "ok"))
                time.sleep(PER_REQ_SLEEP)
            except Exception as e:
                relatorio.append((nome_time, None, 0, 0, 0.0, f"erro:{type(e).__name__}"))

        print(f"✅ Times atualizados: {atualizados} / {len(ids_times)} na coluna '{col_parcial}'")
        for row in relatorio[:12]:
            nm, st, n_atl, n_ok, tot, mot = row
            print(f" • {nm:25} | HTTP={st} | atletas={n_atl:2d} | com_parcial={n_ok:2d} | total={tot:6.2f} | {mot}")
else:
    print("ℹ️ Sem parciais (status != 2). Nenhuma rodada em andamento.")

# Recalcula marcador de líder por rodada (opcional, só se houver algum > 0)
try:
    df_num = df_pontuacoes.apply(pd.to_numeric, errors="coerce")
    if (df_num.sum(numeric_only=True).sum() or 0) != 0:
        lider_rodada = df_num.idxmax()
        df_pontuacoes.loc["Lider_Rodada"] = lider_rodada
except Exception as e:
    print("⚠️ Não foi possível recalcular 'Lider_Rodada':", e)

# Normaliza tipos
df_pontuacoes = df_pontuacoes.apply(pd.to_numeric, errors="coerce").fillna(0.0)

# -------------------------------
# 3) SEGMENTAÇÃO POR TURNOS E MESES
# -------------------------------
rodadas_por_mes = {
    "Janeiro":   [1],
    "Fevereiro": [2, 3, 4],
    "Março":  [5, 6, 7, 8],
    "Abril": [9, 10, 11, 12, 13],
    "Maio": [14, 15, 16, 17, 18],
    # "Junho": [],
    "Julho": [19, 20, 21]
}
rodadas_segundo_turno = list(range(20, 39))

def somar_pontuacoes(df, rodadas):
    colunas = [f'Rodada {i}' for i in rodadas if f'Rodada {i}' in df.columns]
    return df[colunas].drop(index='Lider_Rodada', errors='ignore').sum(axis=1).sort_values(ascending=False)

df_turno_2 = df_pontuacoes[[f'Rodada {i}' for i in rodadas_segundo_turno if f'Rodada {i}' in df_pontuacoes.columns]]
df_soma_turno_2 = somar_pontuacoes(df_pontuacoes, rodadas_segundo_turno)
df_soma_mensal = {mes: somar_pontuacoes(df_pontuacoes, rodadas) for mes, rodadas in rodadas_por_mes.items()}

# -------------------------------
# 4) SALVAR EXCEL E JS
# -------------------------------
with pd.ExcelWriter("Pontuacoes_Liga_Classica_Completa.xlsx") as writer:
    df_pontuacoes.to_excel(writer, sheet_name="Geral")
    df_turno_2.to_excel(writer, sheet_name="Turno 2")
    pd.DataFrame(df_soma_turno_2).to_excel(writer, sheet_name="Classif Turno 2")
    for mes, serie in df_soma_mensal.items():
        pd.DataFrame(serie).to_excel(writer, sheet_name=f"Mês - {mes}")

print("✅ Excel: Pontuacoes_Liga_Classica_Completa.xlsx salvo.")

# JS
df_dict_geral = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').fillna(0).to_dict(orient="index")
df_js = {
    "geral": df_dict_geral,
    "turnos": {"turno_2": df_soma_turno_2.to_dict()},
    "meses": {mes: serie.to_dict() for mes, serie in df_soma_mensal.items()}
}
js_path = "Pontuacoes_Liga_Classica.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoLigaClassica = {json.dumps(df_js, indent=2, ensure_ascii=False)};")
    # Meta (opcional) para o front saber se há parcial e qual rodada é:
    meta = {"status_http": int(status_http), "rodada_parcial": (int(rodada_parcial) if rodada_parcial is not None else None)}
    f.write("\nwindow.ligaClassicaMeta = " + json.dumps(meta) + ";")

print(f"✅ JS: {js_path} salvo.")


2026-01-26 08:33:04,220 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:04,335 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:04,656 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:04,754 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:05,300 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:05,653 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:05,778 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:06,130 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:06,464 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:06,566 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:07,355 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 08:33:07,462 - root -

📌 O campeonato ainda não começou. Criando estrutura com placeholders.
Status HTTP=1 | rodada_http=1 | rodada_api=1 | rodada_parcial=None
ℹ️ Sem parciais (status != 2). Nenhuma rodada em andamento.
✅ Excel: Pontuacoes_Liga_Classica_Completa.xlsx salvo.
✅ JS: Pontuacoes_Liga_Classica.js salvo.


In [9]:
# Exibir a pontuação
display(df_pontuacoes)

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19,Rodada 20,Rodada 21,Rodada 22,Rodada 23,Rodada 24,Rodada 25,Rodada 26,Rodada 27,Rodada 28,Rodada 29,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38
A Lenda Super Vasco F.c,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A Lenda Super Vascão f.c,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AZURRA82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bandoleros FCS,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BordonFC04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C.A. Charru@,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
cartola scheuer17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CARTOLEIRO DO VALLE PRO26.5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
dasdoresfc,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
DM Studio,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
